<a href="https://colab.research.google.com/github/movi-data/painel_performance/blob/main/painel_performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Relatório Plataformas

In [1]:
import pandas as pd

# --- ETAPA 1: Carregar o arquivo ---
df_propostas = pd.read_csv("propostas.csv", sep=";", encoding="utf-8", low_memory=False)

# --- ETAPA 2: Preparar Colunas ---

# Tranformar 'Canal' em maiuscula
df_propostas['Canal'] = df_propostas['Canal'].str.upper()
# Filtrar o primeiro Canal na coluna 'Canal'
df_propostas['Canal'] = df_propostas['Canal'].str.split(',').str[0]

# --- ETAPA 3: Fazer agrupamento e ordernar ---

# Agrupar leads por mês na coluna 'Data' e 'Canal'
leads_por_canal_mes = df_propostas.groupby([pd.to_datetime(df_propostas["Data"], format="%d/%m/%Y", errors='coerce').dt.to_period("d"),
                                            "Canal"]).size().reset_index(name="Quantidade de Leads")
# Ordenar por 'Quantidade de Leads'
leads_por_canal_mes = leads_por_canal_mes.sort_values(by="Quantidade de Leads", ascending=False)

# --- ETAPA 4: Criar arquivo csv e download ---

# Criar arquivo .csv
leads_por_canal_mes.to_csv("leads_por_canal_mes.csv",sep=";" ,index=False)

# Download do arquivo
from google.colab import files
files.download("leads_por_canal_mes.csv")

# Vizualização para conferência
leads_por_canal_mes.head()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Data,Canal,Quantidade de Leads
115,2025-06-11,INSTAGRAM,3013
1207,2025-09-10,INSTAGRAM,2342
148,2025-06-14,INSTAGRAM,2289
910,2025-08-16,WHATSAPP,2264
1195,2025-09-09,INSTAGRAM,2252


In [98]:
import pandas as pd

# --- ETAPA 1: Carregar os arquivos ---
df_propostas = pd.read_csv("propostas.csv", sep=";", encoding="utf-8", low_memory=False)
df_lojas = pd.read_excel("de_para_loja.xlsx")

# --- ETAPA : Preparar Colunas ---
# Tranformar 'Canal' em maiuscula
df_propostas['Canal'] = df_propostas['Canal'].str.upper()
# Filtrar o primeiro Canal na coluna 'Canal'
df_propostas['Canal'] = df_propostas['Canal'].str.split(',').str[0]

# --- ETAPA : Criar coluna 'Bot abandonado' ---
df_propostas['Bot Abandonado'] = df_propostas['tags'].str.contains('Bot Abandonado', case=False)

# --- ETAPA : Novo df com menos colunas ---
df_propostas_resumido = df_propostas[['ID Lead', 'Data', 'Data ultima interação',
                                      'Nome PDV', 'tipo', 'qualificado','Bot Abandonado']]

# --- ETAPA : Criar merge com base no Lojas ---
df_propostas_ajustado = pd.merge(df_propostas_resumido, df_lojas,
                                 on="Nome PDV", how="left", indicator=True)
# --- ETAPA : remover duplicadas ---
df_propostas_ajustado.drop_duplicates(inplace=True)

# --- ETAPA : Criar colunas de data ---

# Converter a coluna Data para datetime
df_propostas_ajustado["Data"] = pd.to_datetime(df_propostas_ajustado["Data"], format="%d/%m/%Y")

# Dia do Mês
df_propostas_ajustado['dia_do_mes'] = df_propostas_ajustado['Data'].dt.day

# Mês do ano (número)
df_propostas_ajustado['mes_do_ano'] = df_propostas_ajustado['Data'].dt.month

# Nome Mês do ano por extenso em português
meses = {
    1: "Janeiro", 2: "Fevereiro", 3: "Março", 4: "Abril",
    5: "Maio", 6: "Junho", 7: "Julho", 8: "Agosto",
    9: "Setembro", 10: "Outubro", 11: "Novembro", 12: "Dezembro"
}
df_propostas_ajustado['nome_mes'] = df_propostas_ajustado['mes_do_ano'].map(meses)

# Dia da semana (número: 0=Segunda, 6=Domingo)
df_propostas_ajustado['dia_da_semana'] = df_propostas_ajustado['Data'].dt.dayofweek

# Nome do dia da semana por extenso em português
dias = {
    0: "Segunda-feira", 1: "Terça-feira", 2: "Quarta-feira",
    3: "Quinta-feira", 4: "Sexta-feira", 5: "Sábado", 6: "Domingo"
}

df_propostas_ajustado['nome_dia'] = df_propostas_ajustado['dia_da_semana'].map(dias)

# Substituir coluna 'Nome PDV' por 'Nome ajustado'
df_propostas_ajustado['Nome PDV'] = df_propostas_ajustado['Nome ajustado']

# Manter somente colunas necessárias
df_propostas_ajustado = df_propostas_ajustado[[

      'ID Lead', 'Data', 'Data ultima interação', 'Nome PDV', 'tipo',
       'qualificado', 'Bot Abandonado', 'CNPJ','Região', 'Varejo', 'UF', 'Cidade',
      'DDD','Status', 'CEP', 'Telefone', 'Endereço ', 'Bairro', 'Mobi ID', 'Regional',
       'dia_do_mes', 'mes_do_ano', 'nome_mes', 'dia_da_semana',
       'nome_dia', '_merge'

]]

# --- ETAPA : Criar arquivo csv e download ---

# Criar arquivo .csv
df_propostas_ajustado.to_csv("propostas_ajustado.csv",sep=";" ,index=False)

# Download do arquivo
from google.colab import files
files.download("propostas_ajustado.csv")

# Vizualização para conferência
df_propostas_ajustado.head()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,ID Lead,Data,Data ultima interação,Nome PDV,tipo,qualificado,Bot Abandonado,CNPJ,Região,Varejo,...,Endereço,Bairro,Mobi ID,Regional,dia_do_mes,mes_do_ano,nome_mes,dia_da_semana,nome_dia,_merge
0,55410293,2025-06-01,01/06/2025,Seminovos Movida Indaiatuba,PERFORMANCE,False,True,21314559022640,R04 - SPI2,Varejo 3,...,Avenida Presidente Vargas 2939 Vila Vitória,Vila Homero,60901,Leonardo Alves Cardoso,1,6,Junho,6,Domingo,both
2,55410294,2025-06-01,17/06/2025,Seminovos Movida Belo Horizonte Raja,PERFORMANCE,True,True,21314559004900,R07 - MG,Varejo 1,...,Avenida Raja Gabaglia 3500 LOJA 1,Estoril,18983,Ari Gomes De Oliveira,1,6,Junho,6,Domingo,both
4,55410295,2025-06-01,01/06/2025,Seminovos Movida Várzea Grande,PERFORMANCE,False,True,21314559018375,R08 - CO,Varejo 1,...,Avenida da Feb 1788,Da Manga,18994,Kaique Silva Gorayeb,1,6,Junho,6,Domingo,both
5,55410296,2025-06-01,01/06/2025,Seminovos Movida Recife Espinheiro,PERFORMANCE,False,True,21314559011281,R09 - NORDESTE,Varejo 1,...,Rua Alfredo de Castro 101,Espinheiro,51076,Wellington Valfrido De Santana,1,6,Junho,6,Domingo,both
7,55410297,2025-06-01,01/06/2025,Seminovos Movida João Pessoa Brisamar,PERFORMANCE,False,True,21314559010986,R09 - NORDESTE,Varejo 1,...,Rua Empresário Clóvis Rolim 2001 Brisamar,Brisamar,58919,Wellington Valfrido De Santana,1,6,Junho,6,Domingo,both


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [95]:
print(df_propostas_ajustado.columns)

Index(['ID Lead', 'Data', 'Data ultima interação', 'Nome PDV', 'tipo',
       'qualificado', 'Bot Abandonado', 'CNPJ', 'Região', 'Varejo', 'UF',
       'Cidade', 'DDD', 'Status', 'CEP', 'Telefone', 'Endereço ', 'Bairro',
       'Mobi ID', 'Regional', 'dia_do_mes', 'mes_do_ano', 'nome_mes',
       'dia_da_semana', 'nome_dia', '_merge'],
      dtype='object')


In [99]:
df_propostas_ajustado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 382270 entries, 0 to 593823
Data columns (total 26 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   ID Lead                382270 non-null  int64         
 1   Data                   382270 non-null  datetime64[ns]
 2   Data ultima interação  382270 non-null  object        
 3   Nome PDV               382270 non-null  object        
 4   tipo                   382270 non-null  object        
 5   qualificado            382270 non-null  bool          
 6   Bot Abandonado         231791 non-null  object        
 7   CNPJ                   380154 non-null  object        
 8   Região                 381681 non-null  object        
 9   Varejo                 381681 non-null  object        
 10  UF                     381681 non-null  object        
 11  Cidade                 381681 non-null  object        
 12  DDD                    381681 non-null  object   

In [97]:
df_propostas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 382253 entries, 0 to 382252
Data columns (total 38 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   ID Lead                           382253 non-null  int64  
 1   Data                              382253 non-null  object 
 2   Data ultima interação             382253 non-null  object 
 3   Nome PDV                          382253 non-null  object 
 4   CNPJ PDV                          382253 non-null  int64  
 5   Nome Cliente                      378900 non-null  object 
 6   Email Cliente                     237292 non-null  object 
 7   Telefone Cliente                  382004 non-null  object 
 8   CPF                               10335 non-null   float64
 9   Lead                              49984 non-null   object 
 10  Origem                            382253 non-null  object 
 11  Canal                             367502 non-null  o

In [107]:
len(df_propostas_ajustado['ID Lead'].unique())


382253

In [108]:
len(df_propostas['ID Lead'].unique())


382253